## Origin Destination Big Data Analysis

In [1]:
import gcsfs as fs
import geopandas as gpd
import numpy as np
import pandas as pd
from calitp_data_analysis import get_fs, utils
from calitp_data_analysis.sql import to_snakecase
from siuba import *

fs = get_fs()

In [2]:
import altair as alt


In [3]:
pd.set_option("display.max_columns", None)


In [4]:
gcs_path = "gs://calitp-analytics-data/data-analyses/big_data/hta/"

In [5]:
from_cp = to_snakecase( pd.read_csv(f"{gcs_path}replica-hta_transit_calpolyhumboldt-10_22_25-trips_dataset_origin_as_CPH.csv"))    

/tmp/ipykernel_464/2358409840.py:1: DtypeWarning: Columns (18,19,20,25,26,28,29,30,31,33,36,38,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  from_cp = to_snakecase( pd.read_csv(f"{gcs_path}replica-hta_transit_calpolyhumboldt-10_22_25-trips_dataset_origin_as_CPH.csv"))


In [6]:
from_cp.sample()

,activity_id,origin_bgrp_2020,origin_trct_2020,origin_cty_2020,origin_st_2020,destination_bgrp_2020,destination_trct_2020,destination_cty_2020,destination_st_2020,primary_mode,trip_purpose,previous_trip_purpose,trip_start_time,trip_end_time,trip_duration_minutes,trip_distance_miles,vehicle_type,vehicle_fuel_type,transit_submode,transit_agency,transit_route,origin_land_use,origin_building_use,destination_land_use,destination_building_use,trip_taker_person_id,trip_taker_household_id,trip_taker_age,trip_taker_sex,trip_taker_race_ethnicity,trip_taker_employment_status,trip_taker_wfh,trip_taker_individual_income,trip_taker_commute_mode,trip_taker_household_size,trip_taker_household_income,trip_taker_available_vehicles,trip_taker_resident_type,trip_taker_industry,trip_taker_building_type,trip_taker_school_grade_attending,trip_taker_education,trip_taker_tenure,trip_taker_language,trip_taker_home_bgrp_2020,trip_taker_home_trct_2020,trip_taker_home_cty_2020,trip_taker_home_st_2020,trip_taker_work_bgrp_2020,trip_taker_work_trct_2020,trip_taker_work_cty_2020,trip_taker_work_st_2020
10066,18261275329581109755,"2 (Tract 10.02, Humboldt, CA)","10.02 (Humboldt, CA)","Humboldt County, CA",California,"3 (Tract 11.02, Humboldt, CA)","11.02 (Humboldt, CA)","Humboldt County, CA",California,walking,eat,school,11:19:55,11:49:56,30,1.5,unknown_vehicle_type,unknown_fuel_type,NaN,NaN,NaN,education,education,mixed_use,retail,10248649262252350861,7853234905192180482,22.0,male,white_not_hispanic_or_latino,employed,employed_not_working,11591.0,private_auto,2,35726.0,zero,core,naics31_33,single_family,undergraduate,some_college,renter,english,"3 (Tract 11.02, Humboldt, CA)","11.02 (Humboldt, CA)","Humboldt County, CA",California,"2 (Tract 12, Humboldt, CA)","12 (Humboldt, CA)","Humboldt County, CA",California


In [7]:
from_cp.origin_bgrp_2020.value_counts()

2 (Tract 10.02, Humboldt, CA)    16623
Name: origin_bgrp_2020, dtype: int64

In [8]:
to_cp = to_snakecase( pd.read_csv(f"{gcs_path}replica-hta_transit_calpolyhumboldt-10_22_25-trips_dataset_dest_as_CPH.csv"))    

/tmp/ipykernel_464/2377608391.py:1: DtypeWarning: Columns (25,26,28,29,30,31,33,36,38,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  to_cp = to_snakecase( pd.read_csv(f"{gcs_path}replica-hta_transit_calpolyhumboldt-10_22_25-trips_dataset_dest_as_CPH.csv"))


In [9]:
to_cp.sample()

,activity_id,origin_bgrp_2020,origin_trct_2020,origin_cty_2020,origin_st_2020,destination_bgrp_2020,destination_trct_2020,destination_cty_2020,destination_st_2020,primary_mode,trip_purpose,previous_trip_purpose,trip_start_time,trip_end_time,trip_duration_minutes,trip_distance_miles,vehicle_type,vehicle_fuel_type,transit_submode,transit_agency,transit_route,origin_land_use,origin_building_use,destination_land_use,destination_building_use,trip_taker_person_id,trip_taker_household_id,trip_taker_age,trip_taker_sex,trip_taker_race_ethnicity,trip_taker_employment_status,trip_taker_wfh,trip_taker_individual_income,trip_taker_commute_mode,trip_taker_household_size,trip_taker_household_income,trip_taker_available_vehicles,trip_taker_resident_type,trip_taker_industry,trip_taker_building_type,trip_taker_school_grade_attending,trip_taker_education,trip_taker_tenure,trip_taker_language,trip_taker_home_bgrp_2020,trip_taker_home_trct_2020,trip_taker_home_cty_2020,trip_taker_home_st_2020,trip_taker_work_bgrp_2020,trip_taker_work_trct_2020,trip_taker_work_cty_2020,trip_taker_work_st_2020
3569,15054122865391088947,"3 (Tract 11.02, Humboldt, CA)","11.02 (Humboldt, CA)","Humboldt County, CA",California,"2 (Tract 10.02, Humboldt, CA)","10.02 (Humboldt, CA)","Humboldt County, CA",California,private_auto,school,home,08:07:00,08:10:01,3,1.1,unknown_vehicle_type,electric,NaN,NaN,NaN,multi_family,multi_family,education,education,9625708196328336841,7945576068312873406,32.0,female,white_not_hispanic_or_latino,employed,employed_not_working,20390.0,private_auto,2,65860.0,two,core,naics61,multiple_units,undergraduate,some_college,renter,english,"3 (Tract 11.02, Humboldt, CA)","11.02 (Humboldt, CA)","Humboldt County, CA",California,"1 (Tract 6, Humboldt, CA)","6 (Humboldt, CA)","Humboldt County, CA",California


In [10]:
to_cp.destination_bgrp_2020.value_counts()

2 (Tract 10.02, Humboldt, CA)    16657
Name: destination_bgrp_2020, dtype: int64

In [11]:
from_cp.primary_mode.value_counts()

private_auto         9179
walking              3686
auto_passenger       3014
biking                289
other_travel_mode     284
commercial             81
on_demand_auto         68
public_transit         22
Name: primary_mode, dtype: int64

In [12]:
to_cp.primary_mode.value_counts()

private_auto         9544
walking              3933
auto_passenger       2457
biking                300
other_travel_mode     254
on_demand_auto         75
commercial             57
public_transit         37
Name: primary_mode, dtype: int64

In [13]:
to_cp.origin_bgrp_2020.value_counts

<bound method IndexOpsMixin.value_counts of 0            1 (Tract 13, Humboldt, CA)
1        2 (Tract 105.03, Humboldt, CA)
2        1 (Tract 105.04, Humboldt, CA)
3           1 (Tract 104, Humboldt, CA)
4           1 (Tract 104, Humboldt, CA)
                      ...              
16652     1 (Tract 10.02, Humboldt, CA)
16653         4 (Tract 2, Humboldt, CA)
16654     1 (Tract 11.03, Humboldt, CA)
16655     2 (Tract 10.02, Humboldt, CA)
16656         3 (Tract 3, Humboldt, CA)
Name: origin_bgrp_2020, Length: 16657, dtype: object>

### Reading in block groups
* downloaded from Replica (also have some other options like taz)

In [14]:
with get_fs().open(f"{gcs_path}replica-hta_transit_calpolyhumboldt-10_22_25-origin_layer_Humb.zip") as f:
    blkgr = to_snakecase(gpd.read_file(f))

In [15]:
blkgr.sample()

,geoname,customgeoid,centroidlat,centroidlon,squaremiles,tripsbyorigin,tripsbyorigin_sqmi,geometry
6309,"1 (Tract 55.08, Sacramento, CA)",60670055081,38.5798,-121.4125,0.076,4,52.6385,"POLYGON ((-121.41533 38.57799, -121.41346 38.5..."


In [16]:
blkgr["name_county"]  = blkgr['geoname'].str.rstrip(', ').str.split(', ').str[1] 

In [17]:
blkgr["name_state"]  = blkgr['geoname'].str.strip().str[-3:-1]

In [18]:
# blkgr = blkgr>>filter(_.name_state=="CA" | _.state_ns)

In [19]:
# blkgr["id_xwalk"] = ""

In [20]:
# blkgr["id_xwalk"] = blkgr['id'].str.lstrip('0')

In [21]:
# blkgr['id_xwalk'] = blkgr['id_xwalk'].astype('int64')

In [22]:
blkgr.sample()

,geoname,customgeoid,centroidlat,centroidlon,squaremiles,tripsbyorigin,tripsbyorigin_sqmi,geometry,name_county,name_state
5671,"1 (Tract 3382.03, Contra Costa, CA)",60133382031,37.9282,-122.0526,0.0438,6,136.9952,"POLYGON ((-122.05464 37.92654, -122.05459 37.9...",Contra Costa,CA


##### read in crosswalk from Replica

In [23]:
# xwalk = pd.read_csv(f"{gcs_path}census_xwalk.csv")
# xwalk = xwalk>>filter(_.stusps=="CA")

In [24]:
# xwalk.sample()

In [25]:
# xwalk_map = dict(zip(xwalk['bgrp'], 
#                           xwalk['bgrpname']))

In [26]:
# blkgr["name_xwalk"] = np.NaN

In [27]:
# blkgr['name_xwalk'] = blkgr['name_xwalk'].fillna(blkgr['id_xwalk'].map(xwalk_map))

In [28]:
to_cp.sample()

,activity_id,origin_bgrp_2020,origin_trct_2020,origin_cty_2020,origin_st_2020,destination_bgrp_2020,destination_trct_2020,destination_cty_2020,destination_st_2020,primary_mode,trip_purpose,previous_trip_purpose,trip_start_time,trip_end_time,trip_duration_minutes,trip_distance_miles,vehicle_type,vehicle_fuel_type,transit_submode,transit_agency,transit_route,origin_land_use,origin_building_use,destination_land_use,destination_building_use,trip_taker_person_id,trip_taker_household_id,trip_taker_age,trip_taker_sex,trip_taker_race_ethnicity,trip_taker_employment_status,trip_taker_wfh,trip_taker_individual_income,trip_taker_commute_mode,trip_taker_household_size,trip_taker_household_income,trip_taker_available_vehicles,trip_taker_resident_type,trip_taker_industry,trip_taker_building_type,trip_taker_school_grade_attending,trip_taker_education,trip_taker_tenure,trip_taker_language,trip_taker_home_bgrp_2020,trip_taker_home_trct_2020,trip_taker_home_cty_2020,trip_taker_home_st_2020,trip_taker_work_bgrp_2020,trip_taker_work_trct_2020,trip_taker_work_cty_2020,trip_taker_work_st_2020
3004,1504855892534747678,"3 (Tract 105.02, Humboldt, CA)","105.02 (Humboldt, CA)","Humboldt County, CA",California,"2 (Tract 10.02, Humboldt, CA)","10.02 (Humboldt, CA)","Humboldt County, CA",California,auto_passenger,school,home,07:50:00,07:57:49,7,5.3,unknown_vehicle_type,unknown_fuel_type,NaN,NaN,NaN,single_family,single_family,education,education,11704714723690785578,9306861264777888105,27.0,female,white_not_hispanic_or_latino,employed,employed_not_working,45704.0,auto_passenger,4,158483.0,two,core,not_working,single_family,not_attending_school,bachelors_degree,renter,english,"3 (Tract 105.02, Humboldt, CA)","105.02 (Humboldt, CA)","Humboldt County, CA",California,"1 (Tract 12, Humboldt, CA)","12 (Humboldt, CA)","Humboldt County, CA",California


In [29]:
# (blkgr[blkgr["name_county"]=="Humboldt"])

In [30]:
# blkgr>>filter(_.name=="1 (Tract 9, Humboldt, CA)")

In [31]:
# (to_cp>>filter(_.origin_bgrp_2020 == "1 (Tract 9, Humboldt, CA)")).sample()

In [32]:
# to_cp.sample()

In [33]:
blkgr_map = dict(zip(blkgr['geoname'], 
                          blkgr['geometry']))

In [34]:
to_cp["origin_geometry"] = np.nan
to_cp['origin_geometry'] = to_cp['origin_geometry'].fillna(to_cp['origin_bgrp_2020'].map(blkgr_map))
to_cp = to_cp.set_geometry("origin_geometry")

In [35]:
# to_cp["dest_geometry"] = np.nan
# to_cp['dest_geometry'] = to_cp['dest_geometry'].fillna(to_cp['destination_bgrp_2020'].map(blkgr_map))

In [36]:
# from_cp["origin_geometry"] = np.nan
# from_cp['origin_geometry'] = from_cp['origin_geometry'].fillna(from_cp['origin_bgrp_2020'].map(blkgr_map))

In [37]:
from_cp["dest_geometry"] = np.nan
from_cp['dest_geometry'] = from_cp['dest_geometry'].fillna(from_cp['destination_bgrp_2020'].map(blkgr_map))

from_cp = from_cp.set_geometry("dest_geometry")

In [38]:
from_cp['origin_bgrp_2020'].unique()

# from_cp.destination_bgrp_2020.value_counts()

array(['2 (Tract 10.02, Humboldt, CA)'], dtype=object)

In [39]:
from_cp_origins = from_cp['destination_bgrp_2020'].unique()

In [40]:
# (blkgr>>filter(_.geoname.isin(from_cp_origins))).explore()

In [41]:
# (blkgr>>filter(_.geoname.isin(from_cp_origins))).explore()

In [42]:
from_cp.sample()

,activity_id,origin_bgrp_2020,origin_trct_2020,origin_cty_2020,origin_st_2020,destination_bgrp_2020,destination_trct_2020,destination_cty_2020,destination_st_2020,primary_mode,trip_purpose,previous_trip_purpose,trip_start_time,trip_end_time,trip_duration_minutes,trip_distance_miles,vehicle_type,vehicle_fuel_type,transit_submode,transit_agency,transit_route,origin_land_use,origin_building_use,destination_land_use,destination_building_use,trip_taker_person_id,trip_taker_household_id,trip_taker_age,trip_taker_sex,trip_taker_race_ethnicity,trip_taker_employment_status,trip_taker_wfh,trip_taker_individual_income,trip_taker_commute_mode,trip_taker_household_size,trip_taker_household_income,trip_taker_available_vehicles,trip_taker_resident_type,trip_taker_industry,trip_taker_building_type,trip_taker_school_grade_attending,trip_taker_education,trip_taker_tenure,trip_taker_language,trip_taker_home_bgrp_2020,trip_taker_home_trct_2020,trip_taker_home_cty_2020,trip_taker_home_st_2020,trip_taker_work_bgrp_2020,trip_taker_work_trct_2020,trip_taker_work_cty_2020,trip_taker_work_st_2020,dest_geometry
13672,7976799408603473666,"2 (Tract 10.02, Humboldt, CA)","10.02 (Humboldt, CA)","Humboldt County, CA",California,"1 (Tract 104, Humboldt, CA)","104 (Humboldt, CA)","Humboldt County, CA",California,private_auto,home,shop,20:11:00,20:20:12,9,7.9,unknown_vehicle_type,other_non_bev,NaN,NaN,NaN,retail,retail,single_family,single_family,1426590177623546588,7724118870784531153,68.0,female,white_not_hispanic_or_latino,not_in_labor_force,unemployed_under_16_not_in_labor_force,0.0,other_travel_mode,1,0.0,one,core,not_working,mobile,not_attending_school,high_school,owner,english,"1 (Tract 104, Humboldt, CA)","104 (Humboldt, CA)","Humboldt County, CA",California,Does not have work/school location,Does not have work/school location,Does not have work/school location,Does not have work/school location,"POLYGON ((-124.12756 40.96310, -124.12532 40.9..."


In [43]:
# from_cp.explore()